In [2]:
import numpy as np


# 4.1

B = np.array([
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
    [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
], dtype=int)


def generate_gogel_code(B, k):
    I = np.eye(k, dtype=int)

    # Формирование матрицы G = [I B]
    G = np.hstack([I, B])

    # Формирование проверочной матрицы H
    H = np.vstack([I, B])

    return G, H


# 4.2

def kill_errors(G, H, count_errors):
    print("Внесение", count_errors, "ошибок в слово")
    word = np.random.randint(0, 2, len(G))
    print('word = ', word)
    code_word = np.dot(word, G) % 2
    print('code_word = ', code_word)
    error_position = np.random.choice(len(G[0] - 1), count_errors, replace=False)
    print('error_position =', error_position)
    code_word_with_error = np.copy(code_word)
    for i in error_position:
        code_word_with_error[i] = (code_word_with_error[i] + 1) % 2
    print("code_word_with_error = ", code_word_with_error)

    syndrome = np.dot(code_word_with_error, H) % 2
    print("syndrome = ", syndrome)
    wt = sum(syndrome)
    print('wt(s) = ', wt)
    if wt <= 3:
        u = np.hstack([syndrome, [0 for _ in range(len(H[0]))]])
        print('u1 = ', u)
        return u
    else:
        u = []
        for i in range(len(B)):
            cur_s = (syndrome + B[i]) % 2
            if sum(cur_s) <= 2:
                e = [0 for _ in range(len(H[0]))]
                e[i] = 1
                u = np.hstack([cur_s, e])
                print('u2 = ', u)
                return u

    second_syndrome = np.dot(syndrome, B) % 2
    print("second_syndrome = ", second_syndrome)
    wt = sum(second_syndrome)
    print('wt(sB) = ', wt)
    if wt <= 3:
        u = np.hstack([[0 for _ in range(len(B[0]))], second_syndrome])
        print('u3 = ', u)
        return u
    else:
        u = []
        for i in range(len(B)):
            cur_s = (second_syndrome + B[i]) % 2
            if sum(cur_s) <= 2:
                e = [0 for _ in range(len(H))]
                e[i] = 1
                u = np.hstack([e, cur_s])
                print('u4 = ', u)
                break
        return u


def study_gogel_code(n, k, d):
    print("Расширенный код Гогеля")
    G, H = generate_gogel_code(B, k)
    print('G = \n', G)
    print('H = \n', H)

    count_errors = 1
    u = kill_errors(G, H, count_errors)
    print('')
    count_errors = 2
    u = kill_errors(G, H, count_errors)
    print('')
    count_errors = 3
    u = kill_errors(G, H, count_errors)
    print('')
    count_errors = 4
    u = kill_errors(G, H, count_errors)
    print('')


n = 24
k = 12
d = 8

study_gogel_code(n, k, d)


# 4.3

def generate_RideMaller_code(r, m):
    if 0 <= r <= m:
        if r == 0:
            G = np.array([1 for _ in range(2 ** m)], dtype=int)
        elif r == m:
            arr = np.array([0 for _ in range(2 ** m - 1)], dtype=int)
            last_str = np.append(arr, 1)
            G = np.vstack([
                generate_RideMaller_code(m - 1, m),
                last_str
            ])
        else:
            first = np.hstack([
                generate_RideMaller_code(r, m - 1),
                generate_RideMaller_code(r, m - 1),
            ])
            last = np.hstack([
                np.array([0 for _ in range(2 ** m - 2 ** (m - 1))], dtype=int),
                generate_RideMaller_code(r - 1, m - 1)
            ])
            G = np.vstack([
                first,
                last
            ])
        return G


# 4.4

def createH(i, m):
    h = np.array([
        [1, 1],
        [1, -1]
    ], dtype=int)
    I_left = np.eye(2 ** (m - i), dtype=int)
    I_rigth = np.eye(2 ** (i - 1), dtype=int)
    H0 = np.kron(I_left,h)
    H = np.kron(H0, I_rigth)
    return H


def change(x):
    if x == 0:
        return -1
    else:
        return x


def kill_errors_in_RideMallerCode(m, G, count_errors):
    print("Внесение", count_errors, "ошибок в слово")
    word = np.random.randint(0, 2, len(G))
    print('word = ', word)
    code_word = np.dot(word, G) % 2
    print('code_word = ', code_word)
    error_position = np.random.choice(len(G[0] - 1), count_errors, replace=False)
    print('error_position =', error_position)
    code_word_with_error = np.copy(code_word)
    for i in error_position:
        code_word_with_error[i] = (code_word_with_error[i] + 1) % 2
    print("code_word_with_error = ", code_word_with_error)
    code_word_with_error = np.array(list(map(lambda x: change(x), code_word_with_error)), dtype=int)
    prev_syndrome = np.copy(code_word_with_error)
    maximum = -1
    pos_maximum = 0
    sign = True
    for i in range(0, m):
        H = createH(i + 1, m)
        print('H(' + str(i + 1) + ',' + str(m) + ') =\n', H)
        syndrome = np.dot(prev_syndrome, H)
        print("w(" + str(i + 1) + ")= ", syndrome)
        prev_syndrome = syndrome
        for i in range(len(syndrome)):
            if maximum < abs(syndrome[i]):
                maximum = abs(syndrome[i])
                pos_maximum = i
                if syndrome[i] >= 0:
                    sign = True
                else:
                    sign = False


    print('maximum =', maximum)
    print('pos_maximum =', pos_maximum)
    v = bin(pos_maximum)[2::][::-1]
    if len(v) < m:
        v += '0' * (m - len(v))
    if sign:
        v = '1' + v
    else:
        v = '0' + v
    print('v = ', v)


def study_RidaMaller_code(r, m):
    print("\nКод Рида-Маллера")
    G = generate_RideMaller_code(r, m)
    print('G(1,3) =\n', G)

    count_errors = 1
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')
    count_errors = 2
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')


r = 1
m = 3
study_RidaMaller_code(r, m)

# 4.5
def full_study_RidaMaller_code(r, m):
    print("\nКод Рида-Маллера \n")
    G = generate_RideMaller_code(r, m)
    print('G(1,4) =\n', G)

    count_errors = 1
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')
    count_errors = 2
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')
    count_errors = 3
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')
    count_errors = 4
    kill_errors_in_RideMallerCode(m, G, count_errors)
    print('')

r = 1
m = 4
full_study_RidaMaller_code(r, m)

Расширенный код Гогеля
G = 
 [[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]]
H = 
 [[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 1 1 1 0 0 0 1 0 1]
 